# Berechnung der Größe der Meteosatpixel

Die Größe der Meteosatpixel nimmt zu, je weiter man sich vom Nadir entfernt. Eine Möglichkeit die Pixelgröße zu ermitteln wäre, die Mittelpunktskoordinaten der Pixel zu nehmen, die Eckpunkte zu bestimmen, und dann diese Unterschiede in km umzurechnen.

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

from analysis_tools import grid_and_interpolation as gi

/vols/talos/local/anaconda2-5.0.0/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Als erstes laden wir die hdf5-Datei mit dem Pixelpunkten.

In [2]:
with h5py.File("/vols/talos/home/stephan/data/SEVIRI/auxdata/msevi-geolocation-rss.h5") as f:
    lon = f['longitude'][:]
    lat = f['latitude'][:]

In [3]:
lon.shape

(3712, 3712)

Die Umrechnung sollte möglich sein, wenn man den Erdradius _r_<sub>e</sub> kennt und die Pixelkoordinaten in das Bogenmaß umrechnet. Mit:

* _r_<sub>e</sub> = 6378 km
* &delta;&phi;<sub>ij</sub>: Breitenvariation des MSG-Pixels ij in Radians
* &delta;&lambda;<sub>ij</sub>: Längenvariation des MSG-Pixels ij in Radians
* &phi;<sub>ij</sub>: Breite des Mittelpunkts des Pixels ij in Radians

kann man die Ausdehnung des Pixels ij berechnen:

* Nord-Süd-Ausdehnung _r_<sub>NS</sub> = _r_<sub>e</sub> &middot; &delta;&phi;<sub>ij</sub>
* Ost-West-Ausdehnung _r_<sub>OW</sub> = _r_<sub>e</sub> &middot; &delta;&lambda;<sub>ij</sub> &middot; cos&phi;<sub>ij</sub>.

Die Fläche des Pixels ergibt sich dann zu:

_A_<sub>ij</sub> = _r_<sub>NS</sub> &middot; _r_<sub>OW</sub> = &delta;&phi;<sub>ij</sub> &middot; &delta;&lambda;<sub>ij</sub> &middot; cos&phi;<sub>ij</sub> &middot; _r_<sub>e</sub><sup>2</sup>

Dann sehen wir uns doch mal ein Beispiel an.

In [9]:
dlon = np.deg2rad(lon[1876,1877] - lon[1876,1876])
print(dlon)

0.00047046406


In [11]:
dlat = np.deg2rad(lat[1875,1876] - lat[1876,1876])
print(dlat)

0.00047367127


In [12]:
la = np.deg2rad(lat[1876,1876])

In [13]:
re = 6378

In [14]:
r_ns = re* dlat
r_ow = re * dlon * np.cos(la)

print (r_ns,r_ow)

(3.0210753768333234, 3.0004850975302384)


In [15]:
A = np.cos(np.deg2rad(la))*dlat*dlon*re**2
print A

9.065098740549331


Das scheint gut zu passen.

Eine weitere Variante wäre, die Pixelgröße aus der Projektion zu ermitteln.

In [17]:
import pyproj

Zuerst legen wir die Satellitenposition und ein paar Gitterparameter fest.

In [20]:
sat_longitudes = {'rss': 9.5,
                  'pzs': 0.0}

sat_parameters = {'std': {'s':3000.40316582,'crow':1856.,'ccol':1856.},
                  'hrv': {'s':1000.1343886066667,'crow':5568.,'ccol':5568.}}

In [21]:
resolution= 'std'
scan_type = 'rss'

In [22]:
s = sat_parameters[resolution]['s']
crow = sat_parameters[resolution]['crow']
ccol = sat_parameters[resolution]['ccol']

Dann definieren wir die Projektion.

In [102]:
proj_msg = pyproj.Proj(proj='geos',
                       h = 35785831,
                       a = 6378169,
                       b = 6356583.8,
                       lon_0 = sat_longitudes[scan_type],
                       units = 'meters',
                       sweep = 'y')

Als nächstes nehmen wir einen Beispielpunkt und rechnen ihn in die Projektionskoordinaten um.

In [103]:
x,y = proj_msg(lon[1856,1856],lat[1856,1856])
print (x,y)

(0.0, 0.0)


In [109]:
print proj_msg(lon[1857,1856],lat[1857,1856])
print proj_msg(lon[1858,1856],lat[1858,1856])
print proj_msg(lon[1856,856],lat[1856,856])

(0.0, -3000.402584965035)
(0.0, -6000.802875231193)
(-3000404.4780611843, 0.0)


In [112]:
print proj_msg(lon[3000,1000],lat[3000,1000])

(-2568331.202740317, -3432442.61319542)


Die ausgegebenen Werte entsprechen der Entfernung vom Ursprung in der jeweiligen Richtung in Metern. Allerdings nur in diskreten 3 km Schritten.

In [105]:
-3000404.4780611843 / s

-1000.0004373549525

In [106]:
5268704.244329075 / s

1755.9987618827738

In [113]:
-3432442.61319542 / s

-1143.993797999258

In [114]:
print("i={},j={}".format(ccol + x / s,crow - y/s))

i=1856.0,j=1856.0


In [108]:
print (x/s, y/s)

(0.0, 0.0)


In [86]:
ccol

1856.0

Was für die Bestimmung der tatsächlichen Pixelprojektionen helfen könnte, ist ein äquidistantes Koordinatensystem.

In [160]:
proj_equi = pyproj.Proj(proj='eqc',
                        lat_ts=0,
                        lon_0=9.5)

In [177]:
pyproj.transform(proj_msg,proj_equi,3000,3000)

(2999.985519102508, 3020.394026933011)

In [178]:
print pyproj.transform(proj_msg,proj_equi,s,s)
print pyproj.transform(proj_msg,proj_equi,-3000404.4780611843, 0.0)

(3000.3886831296804, 3020.7999335444865)
(-3207259.0636816937, 0.0)


In [180]:
3207259.0636816937 / 1e6 

3.207259063681694

In [209]:
def msg_ij2km(i,j):
    x_msg = s * ( i - ccol )
    y_msg = s * ( crow - j )
    
    lo,la = proj_msg(i,j,inverse=True)
    
    x,y = pyproj.transform(proj_msg,proj_equi,x_msg,y_msg)
    
    print ("lon = {}, lat={}\ni = {}, j= {}\nx_msg = {}, y_msg = {}\nx_equ = {}, y_equ = {}".format(lo,la,i,j,x_msg,y_msg,x,y))
    
    if x==0 and y==0:
        x=s
        y=s
        
    x_km = x / 1e6
    y_km = y / 1e6 
    return (x_km,y_km)

In [166]:
print(r"$\lambda=${}, $\phi=${}".format(lon[1856,856],lat[1856,856]))

$\lambda=$-19.3112983704, $\phi=$0.0


In [210]:
msg_ij2km(600,600)

lon = 9.5053898647, lat=0.00542653180953
i = 600, j= 600
x_msg = -3768506.37627, y_msg = 3768506.37627
x_equ = -7039584.58909, y_equ = 4687935.90174


(-7.039584589089902, 4.6879359017363695)

Das sieht nach einem passenden Wert aus. Der MSG-Pixel mit den Koordinaten 1856,856 (hat eine Ausdehnung von 6,45 km in Nord-Süd-Richtung.

In [184]:
(1+s)*x

0.0